In [1]:
%matplotlib inline


In [2]:
import torch
import matplotlib.pyplot as plt
import torchvision
import nengolib
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
from deepsith import DeepSITH
import PIL
from torch.nn.utils import weight_norm

from os.path import join
import scipy.special
import pandas as pd
import seaborn as sn
import scipy
from scipy.spatial.distance import euclidean
from scipy.interpolate import interp1d
from tqdm.notebook import tqdm
import random
from csv import DictWriter
# if gpu is to be used
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor

IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
ttype = FloatTensor

import seaborn as sns
print(use_cuda)
import pickle


AttributeError: module 'time' has no attribute 'clock'

# Load Stimuli

In [ ]:
import collections

def mackey_glass(sample_len=1000, tau=17, delta_t=10, seed=None, n_samples=1):
    # Adapted from https://github.com/mila-iqia/summerschool2015/blob/master/rnn_tutorial/synthetic.py
    '''
    mackey_glass(sample_len=1000, tau=17, seed = None, n_samples = 1) -> input
    Generate the Mackey Glass time-series. Parameters are:
        - sample_len: length of the time-series in timesteps. Default is 1000.
        - tau: delay of the MG - system. Commonly used values are tau=17 (mild 
          chaos) and tau=30 (moderate chaos). Default is 17.
        - seed: to seed the random generator, can be used to generate the same
          timeseries at each invocation.
        - n_samples : number of samples to generate
    '''
    history_len = tau * delta_t 
    # Initial conditions for the history of the system
    timeseries = 1.2
    
    if seed is not None:
        np.random.seed(seed)

    samples = []

    for _ in range(n_samples):
        history = collections.deque(1.2 * np.ones(history_len) + 0.2 * \
                                    (np.random.rand(history_len) - 0.5))
        # Preallocate the array for the time-series
        inp = np.zeros((sample_len,1))
        
        for timestep in range(sample_len):
            for _ in range(delta_t):
                xtau = history.popleft()
                history.append(timeseries)
                timeseries = history[-1] + (0.2 * xtau / (1.0 + xtau ** 10) - \
                             0.1 * history[-1]) / delta_t
            inp[timestep] = timeseries
        
        # Squash timeseries through tanh
        inp = np.tanh(inp - 1)
        samples.append(inp)
    return samples


def generate_data(n_batches, length, split=0.5, seed=0,
                  predict_length=15, tau=17, washout=100, delta_t=1,
                  center=True):
    X = np.asarray(mackey_glass(
        sample_len=length+predict_length+washout, tau=tau,
        seed=seed, n_samples=n_batches))
    X = X[:, washout:, :]
    cutoff = int(split*n_batches)
    if center:
        X -= np.mean(X)  # global mean over all batches, approx -0.066
    Y = X[:, predict_length:, :]
    X = X[:, :-predict_length, :]
    assert X.shape == Y.shape
    return ((X[:cutoff], Y[:cutoff]),
            (X[cutoff:], Y[cutoff:]))

In [ ]:
(train_X, train_Y), (test_X, test_Y) = generate_data(2, 5000)

dataset = torch.utils.data.TensorDataset(torch.Tensor(train_X).cuda(), torch.Tensor(train_Y).cuda())
dataset = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

dataset_valid = torch.utils.data.TensorDataset(torch.Tensor(test_X).cuda(), torch.Tensor(test_Y).cuda())
dataset_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=64, shuffle=False)


## Setup for Model

In [ ]:

def train(model, ttype, train_loader, test_loader, optimizer, loss_func, epoch, 
          loss_buffer_size=800, batch_size=4, device='cuda',
          prog_bar=None, tau=17, pred_len=15):
    
    assert(loss_buffer_size%batch_size==0)
        
    losses = []
    last_test_perf = 0
    best_test_perf = -1
    
    for batch_idx, (data, target) in enumerate(train_loader):
        model.train()
        data = data.to(device).view(data.shape[0],1,1,-1)
        target = target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_func(out,
                         target)

        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().numpy())
        losses = losses[int(-loss_buffer_size/batch_size):]
        
        if ((batch_idx*batch_size)%loss_buffer_size == 0):
            loss_track = {}
            last_test_perf = test_model(model, 'cuda', test_loader, 
                                  )
            loss_track['avg_loss'] = np.mean(losses)
            loss_track['last_test'] = last_test_perf
            loss_track['epoch'] = epoch
            loss_track['batch_idx'] = batch_idx
            loss_track['tau'] = tau
            loss_track['pred_len'] = pred_len
        if not (prog_bar is None):
            # Update progress_bar
            s = "{}:{} Loss: {:.4f}, valid: {:.4f}"
            format_list = [e,batch_idx*batch_size, np.mean(losses), 
                           last_test_perf]         
            s = s.format(*format_list)
            prog_bar.set_description(s)
                
def test_model(model, device, test_loader):
    # Test the Model
    nrmsd = []
    with torch.no_grad():
        for x, y in test_loader:
            data = x.to(device).view(x.shape[0],1,1,-1)
            target = y.to(device)
            optimizer.zero_grad()
            out = model(data)
            nrmsd.append(nengolib.signal.nrmse(out.detach().cpu().numpy().flatten(), 
                                               target=target.detach().cpu().numpy().flatten()))

    perf = np.array(nrmsd).mean()
    return perf

In [ ]:
class DeepSITH_Tracker(nn.Module):
    def __init__(self, out, layer_params, dropout=.5):
        super(DeepSITH_Tracker, self).__init__()
        last_hidden = layer_params[-1]['hidden_size']
        self.hs = DeepSITH(layer_params=layer_params, dropout=dropout)
        self.to_out = nn.Linear(last_hidden, out)
    def forward(self, inp):
        x = self.hs(inp)
        #x = torch.tanh(self.to_out(x))
        x = self.to_out(x)
        return x

In [ ]:
start_tau = 17
start_pd = 15
diffs = [1, 2, 3, 4, 5]
for diff in diffs:
    print(start_tau*diff, start_pd*diff)
    (train_X, train_Y), (test_X, test_Y) = generate_data(128, 5000, 
                                                         tau=start_tau*diff, 
                                                         predict_length=start_pd*diff)

    dataset = torch.utils.data.TensorDataset(torch.Tensor(train_X).cuda(), torch.Tensor(train_Y).cuda())
    dataset = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    dataset_valid = torch.utils.data.TensorDataset(torch.Tensor(test_X).cuda(), torch.Tensor(test_Y).cuda())
    dataset_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=64, shuffle=False)
    
    sith_params1 = {"in_features":1, 
                    "tau_min":1, "tau_max":25.0, 
                    "k":15, 'dt':1,
                    "ntau":8, 'g':0.0,  
                    "ttype":ttype, 'batch_norm':False,
                    "hidden_size":25, "act_func":nn.ReLU()}
    sith_params2 = {"in_features":sith_params1['hidden_size'], 
                    "tau_min":1, "tau_max":50.0, 
                    "k":8, 'dt':1,
                    "ntau":8, 'g':0.0, 
                    "ttype":ttype, 'batch_norm':False,
                    "hidden_size":25, "act_func":nn.ReLU()}
    sith_params3 = {"in_features":sith_params2['hidden_size'], 
                    "tau_min":1, "tau_max":150.0, 'buff_max':600, 
                    "k":4, 'dt':1,
                    "ntau":8, 'g':0.0, 
                    "ttype":ttype, 'batch_norm':False,
                    "hidden_size":25, "act_func":nn.ReLU()}
    layer_params = [sith_params1, sith_params2, sith_params3]

    model = DeepSITH_Tracker(out=1,
                             layer_params=layer_params, 
                             dropout=0.).cuda()
    1/0
    optimizer = torch.optim.Adam(model.parameters(), lr=.004)
    loss_func = nn.MSELoss()

    tot = 0
    for p in model.parameters():
        tot += p.numel()
    print("tot_weights", tot)
    #print(model)
    
    epochs = 1000
    batch_size = 32
    progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
    last_perf = 1000
    for e in progress_bar:
        train(model, ttype, dataset, dataset_valid, 
              optimizer, loss_func, batch_size=batch_size, loss_buffer_size=64,
              epoch=e, perf_file=join('perf','mackeyglass_deepsith_ratio_1.csv'),
              prog_bar=progress_bar, tau=start_tau*diff, pred_len=start_pd*diff)